# Topic extraction using Ollama

In [1]:
from ollama import Client
from pydantic import BaseModel, RootModel
from ollama import chat
import pandas as pd
from tqdm import tqdm
import os

In [2]:
corpus_df = pd.read_csv('data/corpus.csv', index_col=0)

/tmp/ipykernel_8077/1093740753.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  corpus_df = pd.read_csv('data/corpus.csv', index_col=0)


In [3]:
corpus_df.head()

,text,Text_ID,Title,Date,Body,Term,Session,Meeting,Sitting,Agenda,...,Speaker_MP,Speaker_minister,Speaker_party,Speaker_party_name,Party_status,Party_orientation,Speaker_ID,Speaker_name,Speaker_gender,Speaker_birth
ID,,,,,,,,,,,,,,,,,,,,,
ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d7e377,eröffnen 45 Sitzung nationalrates verhindern m...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,Sitting 45,-,...,MP,notMinister,ÖVP,parliamentary group of the Austrian People's P...,-,Centre-right to right,PAD_02822,"Kopf, Karlheinz",M,1957
ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d7e408,hinsichtlich einlangt verhandlungsnständ zuwei...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,Sitting 45,-,...,MP,notMinister,ÖVP,parliamentary group of the Austrian People's P...,-,Centre-right to right,PAD_02822,"Kopf, Karlheinz",M,1957
ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d7e870,Sitzung nationalrates berufen Donnerstag 23 Ok...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,Sitting 45,-,...,MP,notMinister,ÖVP,parliamentary group of the Austrian People's P...,-,Centre-right to right,PAD_02822,"Kopf, Karlheinz",M,1957
ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049_d7e3858,ehrt n en abordnet vizekanzl wünschen schön en...,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,"Sitting Number 49, Legislative period XXV, Wed...",2014-11-19,Lower house,Legislative period XXV,-,-,Sitting 49,-,...,MP,notMinister,SPÖ,parliamentary group of the Social Democratic P...,Coalition,Centre-left,PAD_00145,"Bures, Doris",F,1962
ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049_d7e3889,Sitzung Bundeskanzleramt Vertretung Mitglied B...,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,"Sitting Number 49, Legislative period XXV, Wed...",2014-11-19,Lower house,Legislative period XXV,-,-,Sitting 49,-,...,MP,notMinister,SPÖ,parliamentary group of the Social Democratic P...,Coalition,Centre-left,PAD_00145,"Bures, Doris",F,1962


In [4]:
num_unique = corpus_df['Text_ID'].nunique()
print(f"Number of unique Text_IDs: {num_unique}")

Number of unique Text_IDs: 1221


# Use Ollama model for classification

In [49]:
doc_content = ""
# meta prompt
META_PROMPT = """
You are a topic classification assistant. Your task is to analyze the following document and identify **exactly three** relevant topics from the predefined list of public policy categories. Each selected topic must reflect a *core thematic focus* of the document — choose the best semantic match even if the topic isn't a perfect fit.

**Rules:**
- Use only the predefined list below. Do not invent new topics.
- You must return exactly 3 topics. Never return more or fewer.
- If the fit is imperfect, choose the nearest applicable topic and explain why.
- Use confidence scores between 0 and 1 to reflect your certainty. Do not assign the same value to all three.
- Your output must be a single JSON array — do not add commentary or explanations outside the JSON.
- Use only the exact topics from the predefined list. Use only the exact topic names as written. Do not paraphrase, abbreviate, or alter them.

### Predefined Topics:
    Hospital Funding Models (federal vs. provincial responsibilities)
    Health Insurance System (ÖGK and other insurers)
    Medical Workforce Shortage (rural access and doctor quotas)
    Curriculum Reform (digital skills, civic education)
    Teacher Recruitment and Pay (regional disparities)
    Inclusive Education (support for children with disabilities or language barriers)
    University Autonomy (freedom in hiring and budgeting)
    Research Funding Allocation (FFG, FWF grant systems)
    Access and Tuition (student fees for non-EU nationals)
    Conscription Reform (debate over compulsory service)
    Cybersecurity Capabilities (military defense in the digital space)
    Disaster Relief Operations (Bundesheer role in floods, wildfires)
    Counterterrorism Laws (post-2020 Vienna attack measures)
    Police Use of Force (body cams, accountability mechanisms)
    Border Control and Migration Policing (EU Schengen concerns)
    Judicial Independence (selection of judges and prosecutors)
    Anti-Corruption Enforcement (high-profile political cases)
    Legal Aid and Access to Justice (for low-income citizens)
    Tax Reform (income tax brackets, corporate tax adjustments)
    Public Debt Management (compliance with EU fiscal rules)
    Digital Taxation (taxing tech giants like Google and Amazon)
    Austria's Neutrality (debates around NATO relations)
    EU Enlargement Policy (position on Balkans integration)
    Development Cooperation (aid to Africa and conflict zones)
    Minimum Wage Debates (sectoral collective agreements)
    Short-Time Work Programs (Kurzarbeit, especially post-COVID)
    Skilled Labor Immigration (Red-White-Red Card system)
    Pension Age and Sustainability (aging population pressure)
    Family Allowances and Child Benefits (Familienbeihilfe)
    Basic Income Proposals (pilot discussions and party platforms)
    Carbon Pricing System (CO₂ tax and national emissions trading)
    Nature Conservation Laws (protection of Alpine biodiversity)
    Waste Management and Circular Economy (plastic bans, recycling targets)
    Renewable Energy Expansion (wind, solar, hydro)
    Energy Grid Modernization (smart grid and interconnectivity)
    Public Transport Investment (ÖBB modernization, rural coverage)
    EU Agricultural Subsidies (CAP negotiations)
    Sustainable Farming Practices (organic and climate-resilient crops)
    Forest Management and Wildfire Prevention (role of state forests)
    Urban-Rural Infrastructure Gap (digital access, transit)
    Local Government Financing (municipal tax sharing)
    Housing Policy at Local Level (affordable housing mandates)
    Public Broadcasting Regulation (ORF independence and funding)
    Support for Creative Industries (COVID-era relief for artists)
    Cultural Heritage Preservation (restoration and tourism policy)
    Parliamentary Oversight and Investigative Committees
    Legislative Procedure and Transparency
    Political Accountability and Scandals

### Now classify this document:
{doc_content}

### Output Format (Exactly 3 Entries Required):
```json
[
  {
    "topic": "Anti-Corruption Enforcement",
    "confidence": 0.95,
    "explanation": "The core of the session involved multiple motions tied to findings and actions resulting from a parliamentary investigation into surveillance and undue influence—directly pointing to corruption control and political misconduct."
  },
  {
    "topic": "Judicial Independence",
    "confidence": 0.77,
    "explanation": "While not a court case per se, the discussion around investigation timelines, committee oversight, and procedural justice touches on institutional checks and the legal-political interface."
  },
  {
    "topic": "Legal Aid and Access to Justice",
    "confidence": 0.52,
    "explanation": "Weak fit, but justified only because some debates in the session may concern whether political actors and citizens alike have adequate institutional recourse in scandals involving surveillance and misconduct."
  }
]

If the document covers more than 3 relevant themes, select the 3 that are most central to its content. If you identify fewer than 3 highly relevant topics, still return exactly 3 by selecting the next closest applicable topics and clearly justify their inclusion.

"""

## Test if connection works with random file

In [50]:
file_path = "./data/parlamint_at_extracted/ParlaMint-AT.txt/2009/ParlaMint-AT_2009-12-11-024-XXIV-NRSITZ-00050.txt"

ollama_client = Client(host='http://localhost:11434')

with open(file_path) as f:
    s = f.read()
    full_prompt = META_PROMPT.replace("{doc_content}", s)

    response = ollama_client.chat(
        model="phi3",
        messages=[{'role': 'user', 'content': full_prompt}]
    )

print(response['message']['content'])


```json
[
  {
    "topic": "Anti-Corruption Enforcement",
    "confidence": 0.95,
    "explanation": "The National Council session was heavily focused on anti-corruption measures following the findings of a parliamentary committee investigating abuse and surveillance in Parliament's procedures—highlighting political integrity as paramount."
  },
  
  {
    "topic": "Judicial Independence",
    "confidence": 0.82,
    "explanation": "The discussions around oversight and legal proceedings in the parliamentary setting suggest a strong interest in how justice mechanisms can uphold or fail judicial independence within political structures—a secondary but related theme."
  },
  
  {
    "topic": "Legal Aid and Access to Justice",
    "confidence": 0.52,
    "explanation": "Though the direct relevance is less pronounced in this parliamentary session content, accessibility issues surrounding justice were tangentially present due to questions on how citizens can seek recourse for misconduct—mak

## Test it with a schema for consistence responses

In [51]:
class TopicEntry(BaseModel):
    topic: str
    confidence: float
    explanation: str

class TopicsList(RootModel):
    root: list[TopicEntry]

schema = TopicsList.model_json_schema()

response = chat(
    messages=[{'role':'user', 'content': full_prompt}],
    model='llama3',  # or another
    format=schema
)

data = TopicsList.model_validate_json(response.message.content)

In [53]:
data

TopicsList(root=[TopicEntry(topic='Anti-Corruption Enforcement', confidence=0.95, explanation='The core of the session involved multiple motions tied to findings and actions resulting from a parliamentary investigation into surveillance and undue influence—directly pointing to corruption control and political misconduct.'), TopicEntry(topic='Judicial Independence', confidence=0.77, explanation='While not a court case per se, the discussion around investigation timelines, committee oversight, and procedural justice touches on institutional checks and the legal-political interface.'), TopicEntry(topic='Government Transparency and Accountability', confidence=0.62, explanation='The session highlights concerns about secrecy and lack of transparency in government operations, emphasizing the need for greater accountability and legislative scrutiny.')])

In [257]:
ollama_client = Client(host='http://localhost:11434')

def query_ollama(prompt: str) -> TopicsList:
    try:
        response = ollama_client.chat(
            messages=[{'role': 'user', 'content': prompt}],
            model='llama3',
            format=schema,
            options={"timeout": 30},
        )
        validated = TopicsList.model_validate_json(response['message']['content'])

        if len(validated.root) > 3:
            # Use model_copy to create a new validated object with sliced root
            validated = validated.model_copy(update={"root": validated.root[:3]})
        elif len(validated.root) < 3:
            print(validated)
            raise ValueError(f"Only {len(validated.root)} topics returned")

        return validated

    except Exception as e:
        print(validated)
        raise RuntimeError(f"Error during classification: {e}")



In [55]:
base_path = "./data/parlamint_at_extracted/ParlaMint-AT.txt"

# gather all .txt files
base = "./data/parlamint_at_extracted/ParlaMint-AT.txt"
valid_years = {"2019", "2020", "2021", "2022"}
all_txt_files = [
    os.path.join(base, yr, f)
    for yr in os.listdir(base)
    if yr in valid_years and os.path.isdir(os.path.join(base, yr))
    for f in os.listdir(os.path.join(base, yr))
    if f.endswith(".txt")
]

print(f"Found {len(all_txt_files)} .txt files.")

Found 208 .txt files.


# Classify

In [58]:
result_rows = []

for path in tqdm(all_txt_files, desc="Processing files"):
    file = os.path.basename(path)
    with open(path, 'r', encoding='utf-8') as f:
        content = f.read()

    full_prompt = META_PROMPT.replace("{doc_content}", content)

    try:
        topics = query_ollama(prompt=full_prompt)
        # print(topics)
    except Exception as e:
        print(f"[X] Failed to classify {file}: {e}")
        continue

    if not topics or len(topics.root) != 3:
        print(f"[!] Unexpected topic format in {file}")
        continue

    filename_base = file.replace(".txt", "")
    text_id = filename_base.split("_d")[0]

    row = {"Text_ID": text_id}
    for i, entry in enumerate(topics.root, 1):
        row[f"Topic {i}"] = entry.topic
        row[f"Confidence {i}"] = entry.confidence

    result_rows.append(row)

# Build DataFrame
classified_df = pd.DataFrame(result_rows, columns=[
    "Text_ID",
    "Topic 1", "Confidence 1",
    "Topic 2", "Confidence 2",
    "Topic 3", "Confidence 3"
])


Processing files:  12%|█▏        | 25/208 [02:00<13:59,  4.59s/it]

[X] Failed to classify ParlaMint-AT_2021-03-26-027-XXVII-NRSITZ-00093.txt: Error during classification: Only 0 topics returned


Processing files:  15%|█▌        | 32/208 [02:36<13:36,  4.64s/it]

[X] Failed to classify ParlaMint-AT_2021-04-21-027-XXVII-NRSITZ-00097.txt: Error during classification: Only 0 topics returned


Processing files:  25%|██▌       | 53/208 [04:31<12:06,  4.69s/it]

...

Processing files:  63%|██████▎   | 131/208 [11:16<05:39,  4.41s/it]

[X] Failed to classify ParlaMint-AT_2020-07-09-027-XXVII-NRSITZ-00047.txt: Error during classification: Only 0 topics returned


Processing files:  96%|█████████▌| 200/208 [17:37<00:42,  5.31s/it]

[X] Failed to classify ParlaMint-AT_2019-05-15-026-XXVI-NRSITZ-00074.txt: Error during classification: Only 0 topics returned


Processing files: 100%|██████████| 208/208 [18:17<00:00,  5.28s/it]


In [ ]:
classified_df.to_csv("./data/classified_df.csv", index=False)

# Validate if the classification worked and the df can be merged

In [70]:
classified_df

,Text_ID,Topic 1,Confidence 1,Topic 2,Confidence 2,Topic 3,Confidence 3
0,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Parliamentary Oversight and Investigative Comm...,0.62
1,ParlaMint-AT_2021-05-17-027-XXVII-NRSITZ-00103,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Parliamentary Oversight,0.62
2,ParlaMint-AT_2021-07-07-027-XXVII-NRSITZ-00116,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Legislative Procedure and Transparency,0.42
3,ParlaMint-AT_2021-05-19-027-XXVII-NRSITZ-00106,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Legislative Procedure and Transparency,0.43
4,ParlaMint-AT_2021-03-25-027-XXVII-NRSITZ-00092,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Legislative Procedure and Transparency,0.62
...,...,...,...,...,...,...,...
195,ParlaMint-AT_2019-05-27-026-XXVI-NRSITZ-00079,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Political Accountability,0.65
196,ParlaMint-AT_2019-01-30-026-XXVI-NRSITZ-00062,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Legislative Procedure and Transparency,0.42
197,ParlaMint-AT_2019-04-24-026-XXVI-NRSITZ-00070,Rechtsschutzlücken im Amtshaftungsrecht,0.87,Judicial Independence,0.74,Legal Aid and Access to Justice,0.44
198,ParlaMint-AT_2019-07-02-026-XXVI-NRSITZ-00084,Education,0.95,Government Policy,0.85,Social Justice,0.75


In [69]:
classified_df.count()

Text_ID         200
Topic 1         200
Confidence 1    200
Topic 2         200
Confidence 2    200
Topic 3         200
Confidence 3    200
dtype: int64

In [91]:
classified_ids = set(classified_df['Text_ID'].str.strip().unique())
corpus_ids = set(corpus_df['Text_ID'].str.strip().unique())

missing_ids = classified_ids - corpus_ids
print(f"{len(missing_ids)} classified Text_IDs not found in corpus_df")
print("Examples:", list(missing_ids)[:5])

0 classified Text_IDs not found in corpus_df
Examples: []


In [92]:
print("Total corpus_df rows with Topic 1 assigned:", corpus_df["Topic 1"].notna().sum())
print("Expected minimum (should be > 200):", classified_df["Text_ID"].nunique())

Total corpus_df rows with topic_1 assigned: 28845
Expected minimum (should be > 200): 200


In [93]:
corpus_df = corpus_df.merge(classified_df, on="Text_ID", how="left")

In [94]:
print("Unique Text_IDs in corpus_df:", corpus_df['Text_ID'].nunique())
print("Unique Text_IDs in classified_df:", classified_df['Text_ID'].nunique())

Unique Text_IDs in corpus_df: 1221
Unique Text_IDs in classified_df: 200


In [95]:
corpus_df

,index,ID,text,Text_ID,Title,Date,Body,Term,Session,Meeting,...,Topic 2_x,Confidence 2_x,Topic 3_x,Confidence 3_x,Topic 1_y,Confidence 1_y,Topic 2_y,Confidence 2_y,Topic 3_y,Confidence 3_y
0,0,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,eröffnen 45 Sitzung nationalrates verhindern m...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,hinsichtlich einlangt verhandlungsnständ zuwei...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,Sitzung nationalrates berufen Donnerstag 23 Ok...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049_d...,ehrt n en abordnet vizekanzl wünschen schön en...,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,"Sitting Number 49, Legislative period XXV, Wed...",2014-11-19,Lower house,Legislative period XXV,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049_d...,Sitzung Bundeskanzleramt Vertretung Mitglied B...,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,"Sitting Number 49, Legislative period XXV, Wed...",2014-11-19,Lower house,Legislative period XXV,-,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231408,231408,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066_...,davor mögen weidinr Wort melden bitte,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066,"Sitting Number 66, Legislative period XXVI, We...",2019-03-27,Lower house,Legislative period XXVI,-,-,...,Parliamentary Procedure,0.81,Democratization,0.68,Government Transparency,0.93,Parliamentary Procedure,0.81,Democratization,0.68
231409,231409,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066_...,schätzen wert Kolleginn wert kollen wissen Hau...,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066,"Sitting Number 66, Legislative period XXVI, We...",2019-03-27,Lower house,Legislative period XXVI,-,-,...,Parliamentary Procedure,0.81,Democratization,0.68,Government Transparency,0.93,Parliamentary Procedure,0.81,Democratization,0.68
231410,231410,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066_...,Wort niemand melden Debatte schließen weisen A...,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066,"Sitting Number 66, Legislative period XXVI, We...",2019-03-27,Lower house,Legislative period XXVI,-,-,...,Parliamentary Procedure,0.81,Democratization,0.68,Government Transparency,0.93,Parliamentary Procedure,0.81,Democratization,0.68
231411,231411,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066_...,Abstimmung Antrag Abordnet ottenschlär Plessl ...,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066,"Sitting Number 66, Legislative period XXVI, We...",2019-03-27,Lower house,Legislative period XXVI,-,-,...,Parliamentary Procedure,0.81,Democratization,0.68,Government Transparency,0.93,Parliamentary Procedure,0.81,Democratization,0.68


In [121]:
corpus_df[corpus_df["Text_ID"] == "ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088"]

,index,ID,text,Text_ID,Title,Date,Body,Term,Session,Meeting,...,Speaker_ID,Speaker_name,Speaker_gender,Speaker_birth,Topic 1,Confidence 1,Topic 2,Confidence 2,Topic 3,Confidence 3
163732,163732,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088...,eröffnen 88 Sitzung nationalrates verhindern m...,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088,"Sitting Number 88, Legislative period XXVII, M...",2021-03-08,Lower house,Legislative period XXVII,-,-,...,PAD_00145,"Bures, Doris",F,1962,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Parliamentary Oversight and Investigative Comm...,0.62
163733,163733,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088...,hinsichtlich einlangt verhandlungsnständ zuwei...,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088,"Sitting Number 88, Legislative period XXVII, M...",2021-03-08,Lower house,Legislative period XXVII,-,-,...,PAD_00145,"Bures, Doris",F,1962,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Parliamentary Oversight and Investigative Comm...,0.62
163734,163734,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088...,s weisen Antrag 1397 a Kulturausschuss anträ 1...,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088,"Sitting Number 88, Legislative period XXVII, M...",2021-03-08,Lower house,Legislative period XXVII,-,-,...,PAD_00145,"Bures, Doris",F,1962,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Parliamentary Oversight and Investigative Comm...,0.62
163735,163735,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088...,Eingang Tasordnung teilen abordnet Keck Kolleg...,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088,"Sitting Number 88, Legislative period XXVII, M...",2021-03-08,Lower house,Legislative period XXVII,-,-,...,PAD_00145,"Bures, Doris",F,1962,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Parliamentary Oversight and Investigative Comm...,0.62
163736,163736,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088...,Sitzung nationalrates Mittwoch 24 März 2021 9 ...,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088,"Sitting Number 88, Legislative period XXVII, M...",2021-03-08,Lower house,Legislative period XXVII,-,-,...,PAD_00145,"Bures, Doris",F,1962,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Parliamentary Oversight and Investigative Comm...,0.62


In [120]:
classified_df[classified_df["Text_ID"] == "ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088"]

,Text_ID,Topic 1,Confidence 1,Topic 2,Confidence 2,Topic 3,Confidence 3
0,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Parliamentary Oversight and Investigative Comm...,0.62


In [30]:
corpus_df["Topic 1"].value_counts().reset_index(name='Count').rename(columns={'index': 'Topic'})

,Topic 1,Count
0,Anti-Corruption Enforcement (high-profile poli...,58021
1,Political Accountability and Scandals,32408
2,Parliamentary Oversight and Investigative Comm...,28369
3,Legislative Procedure and Transparency,15179
4,Public Transport Investment,12726
5,"Tax Reform (income tax brackets, corporate tax...",11812
6,"Curriculum Reform (digital skills, civic educa...",6022
7,Housing Policy at Local Level,4818
8,Waste Management and Circular Economy (plastic...,4628
9,Local Government Financing,4222


In [31]:
corpus_df["Topic 2"].value_counts().reset_index(name='Count').rename(columns={'index': 'Topic'})

,Topic 2,Count
0,Parliamentary Oversight and Investigative Comm...,101312
1,Political Accountability and Scandals,30147
2,Public Transport Investment,19109
3,"Tax Reform (income tax brackets, corporate tax...",11770
4,Legislative Procedure and Transparency,11383
5,Development Cooperation (aid to Africa and con...,7209
6,Local Government Financing,6182
7,"Curriculum Reform (digital skills, civic educa...",4629
8,Minimum Wage Debates (sectoral collective agre...,3394
9,University Autonomy (freedom in hiring and bud...,3248


In [32]:
corpus_df["Topic 3"].value_counts().reset_index(name='Count').rename(columns={'index': 'Topic'})

,Topic 3,Count
0,Parliamentary Oversight and Investigative Comm...,77270
1,Political Accountability and Scandals,53553
2,Legislative Procedure and Transparency,21441
3,Public Transport Investment,17138
4,"Tax Reform (income tax brackets, corporate tax...",7608
5,Development Cooperation (aid to Africa and con...,7239
6,"Curriculum Reform (digital skills, civic educa...",4503
7,Local Government Financing,3415
8,Cybersecurity Capabilities (military defense i...,3242
9,Legal Aid and Access to Justice (for low-incom...,3105


In [115]:
corpus_df['Topic 1'].notna().sum()


np.int64(28845)

In [122]:
corpus_df.to_csv('./data/corpus_topics.csv', index=False)

# Classify more data

this finds all the files that have not been classified yet. for easier rerun or pausing etc.

In [151]:
# load corpus if necessary
corpus_df = pd.read_csv('./data/corpus_topics.csv')
corpus_df.head()

/tmp/ipykernel_1357/1550299397.py:2: DtypeWarning: Columns (24,25,27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  corpus_df = pd.read_csv('./data/corpus_topics.csv')


,index,ID,text,Text_ID,Title,Date,Body,Term,Session,Meeting,...,Speaker_ID,Speaker_name,Speaker_gender,Speaker_birth,Topic 1,Confidence 1,Topic 2,Confidence 2,Topic 3,Confidence 3
0,0,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,eröffnen 45 Sitzung nationalrates verhindern m...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,...,PAD_02822,"Kopf, Karlheinz",M,1957,NaN,NaN,NaN,NaN,NaN,NaN
1,1,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,hinsichtlich einlangt verhandlungsnständ zuwei...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,...,PAD_02822,"Kopf, Karlheinz",M,1957,NaN,NaN,NaN,NaN,NaN,NaN
2,2,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,Sitzung nationalrates berufen Donnerstag 23 Ok...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,...,PAD_02822,"Kopf, Karlheinz",M,1957,NaN,NaN,NaN,NaN,NaN,NaN
3,3,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049_d...,ehrt n en abordnet vizekanzl wünschen schön en...,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,"Sitting Number 49, Legislative period XXV, Wed...",2014-11-19,Lower house,Legislative period XXV,-,-,...,PAD_00145,"Bures, Doris",F,1962,NaN,NaN,NaN,NaN,NaN,NaN
4,4,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049_d...,Sitzung Bundeskanzleramt Vertretung Mitglied B...,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,"Sitting Number 49, Legislative period XXV, Wed...",2014-11-19,Lower house,Legislative period XXV,-,-,...,PAD_00145,"Bures, Doris",F,1962,NaN,NaN,NaN,NaN,NaN,NaN


In [167]:
corpus_df.head()

,index,ID,text,Text_ID,Title,Date,Body,Term,Session,Meeting,...,Speaker_ID,Speaker_name,Speaker_gender,Speaker_birth,Topic 1,Confidence 1,Topic 2,Confidence 2,Topic 3,Confidence 3
0,0,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,eröffnen 45 Sitzung nationalrates verhindern m...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,...,PAD_02822,"Kopf, Karlheinz",M,1957,NaN,NaN,NaN,NaN,NaN,NaN
1,1,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,hinsichtlich einlangt verhandlungsnständ zuwei...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,...,PAD_02822,"Kopf, Karlheinz",M,1957,NaN,NaN,NaN,NaN,NaN,NaN
2,2,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,Sitzung nationalrates berufen Donnerstag 23 Ok...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,...,PAD_02822,"Kopf, Karlheinz",M,1957,NaN,NaN,NaN,NaN,NaN,NaN
3,3,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049_d...,ehrt n en abordnet vizekanzl wünschen schön en...,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,"Sitting Number 49, Legislative period XXV, Wed...",2014-11-19,Lower house,Legislative period XXV,-,-,...,PAD_00145,"Bures, Doris",F,1962,NaN,NaN,NaN,NaN,NaN,NaN
4,4,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049_d...,Sitzung Bundeskanzleramt Vertretung Mitglied B...,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,"Sitting Number 49, Legislative period XXV, Wed...",2014-11-19,Lower house,Legislative period XXV,-,-,...,PAD_00145,"Bures, Doris",F,1962,NaN,NaN,NaN,NaN,NaN,NaN


In [256]:
base = "./data/parlamint_at_extracted/ParlaMint-AT.txt"
valid_years = {str(y) for y in range(1996, 2005)}

already_classified_ids = set(
    corpus_df.dropna(subset=['Topic 1', 'Topic 2', 'Topic 3'])['Text_ID'].unique()
)

all_txt_files = []

for yr in os.listdir(base):
    year_path = os.path.join(base, yr)
    if yr not in valid_years or not os.path.isdir(year_path):
        continue

    for f in os.listdir(year_path):
        if not f.endswith(".txt"):
            continue

        file_path = os.path.join(year_path, f)
        text_id = os.path.splitext(f)[0]

        if text_id not in already_classified_ids:
            all_txt_files.append(file_path)

print(f"Found {len(all_txt_files)} .txt files for classification.")


Found 0 .txt files for classification.


In [258]:
result_rows = []

for path in tqdm(all_txt_files, desc="Processing files"):
    file = os.path.basename(path)
    with open(path, 'r', encoding='utf-8') as f:
        content = f.read()

    full_prompt = META_PROMPT.replace("{doc_content}", content)

    try:
        topics = query_ollama(prompt=full_prompt)
        # print(topics)
    except Exception as e:
        #print(f"[X] Failed to classify {file}: {e}")
        continue

    if not topics or len(topics.root) != 3:
        #print(f"[!] Unexpected topic format in {file}")
        continue

    filename_base = file.replace(".txt", "")
    text_id = filename_base.split("_d")[0]

    row = {"Text_ID": text_id}
    for i, entry in enumerate(topics.root, 1):
        row[f"Topic {i}"] = entry.topic
        row[f"Confidence {i}"] = entry.confidence

    result_rows.append(row)

# Build DataFrame
classified_df = pd.DataFrame(result_rows, columns=[
    "Text_ID",
    "Topic 1", "Confidence 1",
    "Topic 2", "Confidence 2",
    "Topic 3", "Confidence 3"
])


Processing files: 0it [00:00, ?it/s]


In [259]:
classified_df

,Text_ID,Topic 1,Confidence 1,Topic 2,Confidence 2,Topic 3,Confidence 3


In [260]:
classified_ids = set(classified_df['Text_ID'].str.strip().unique())
corpus_ids = set(corpus_df['Text_ID'].str.strip().unique())

missing_ids = classified_ids - corpus_ids
print(f"{len(missing_ids)} classified Text_IDs not found in corpus_df")
print("Examples:", list(missing_ids)[:5])

0 classified Text_IDs not found in corpus_df
Examples: []


In [261]:
merged = pd.merge(
    corpus_df,
    classified_df,
    on="Text_ID",
    how="left",
    suffixes=('', '_new')  # avoid _x, _y mess
)

for col in ['Topic 1', 'Confidence 1', 'Topic 2', 'Confidence 2', 'Topic 3', 'Confidence 3']:
    new_col = f"{col}_new"
    if col in merged.columns and new_col in merged.columns:
        merged[col] = merged[col].fillna(merged[new_col])
        merged.drop(columns=[new_col], inplace=True)

corpus_df = merged


In [262]:
print("Unique Text_IDs in corpus_df:", corpus_df['Text_ID'].nunique())
print("Unique Text_IDs in classified_df:", classified_df['Text_ID'].nunique())

Unique Text_IDs in corpus_df: 1221
Unique Text_IDs in classified_df: 0


In [263]:
corpus_df

,index,ID,text,Text_ID,Title,Date,Body,Term,Session,Meeting,...,Speaker_ID,Speaker_name,Speaker_gender,Speaker_birth,Topic 1,Confidence 1,Topic 2,Confidence 2,Topic 3,Confidence 3
0,0,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,eröffnen 45 Sitzung nationalrates verhindern m...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,...,PAD_02822,"Kopf, Karlheinz",M,1957,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Legal Aid and Access to Justice,0.52
1,1,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,hinsichtlich einlangt verhandlungsnständ zuwei...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,...,PAD_02822,"Kopf, Karlheinz",M,1957,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Legal Aid and Access to Justice,0.52
2,2,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,Sitzung nationalrates berufen Donnerstag 23 Ok...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,...,PAD_02822,"Kopf, Karlheinz",M,1957,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Legal Aid and Access to Justice,0.52
3,3,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049_d...,ehrt n en abordnet vizekanzl wünschen schön en...,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,"Sitting Number 49, Legislative period XXV, Wed...",2014-11-19,Lower house,Legislative period XXV,-,-,...,PAD_00145,"Bures, Doris",F,1962,Family Welfare,0.95,Government Reform,0.78,Legislation and Policy,0.62
4,4,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049_d...,Sitzung Bundeskanzleramt Vertretung Mitglied B...,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,"Sitting Number 49, Legislative period XXV, Wed...",2014-11-19,Lower house,Legislative period XXV,-,-,...,PAD_00145,"Bures, Doris",F,1962,Family Welfare,0.95,Government Reform,0.78,Legislation and Policy,0.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231408,231408,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066_...,davor mögen weidinr Wort melden bitte,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066,"Sitting Number 66, Legislative period XXVI, We...",2019-03-27,Lower house,Legislative period XXVI,-,-,...,PAD_00145,"Bures, Doris",F,1962,Government Transparency,0.93,Parliamentary Procedure,0.81,Democratization,0.68
231409,231409,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066_...,schätzen wert Kolleginn wert kollen wissen Hau...,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066,"Sitting Number 66, Legislative period XXVI, We...",2019-03-27,Lower house,Legislative period XXVI,-,-,...,PAD_02331,"Weidinger, Peter",M,1977,Government Transparency,0.93,Parliamentary Procedure,0.81,Democratization,0.68
231410,231410,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066_...,Wort niemand melden Debatte schließen weisen A...,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066,"Sitting Number 66, Legislative period XXVI, We...",2019-03-27,Lower house,Legislative period XXVI,-,-,...,PAD_00145,"Bures, Doris",F,1962,Government Transparency,0.93,Parliamentary Procedure,0.81,Democratization,0.68
231411,231411,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066_...,Abstimmung Antrag Abordnet ottenschlär Plessl ...,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066,"Sitting Number 66, Legislative period XXVI, We...",2019-03-27,Lower house,Legislative period XXVI,-,-,...,PAD_00145,"Bures, Doris",F,1962,Government Transparency,0.93,Parliamentary Procedure,0.81,Democratization,0.68


In [264]:
corpus_df.to_csv('./data/corpus_topics.csv', index=False)

In [166]:
print(corpus_df.columns.tolist())


['index', 'ID', 'text', 'Text_ID', 'Title', 'Date', 'Body', 'Term', 'Session', 'Meeting', 'Sitting', 'Agenda', 'Subcorpus', 'Lang', 'Speaker_role', 'Speaker_MP', 'Speaker_minister', 'Speaker_party', 'Speaker_party_name', 'Party_status', 'Party_orientation', 'Speaker_ID', 'Speaker_name', 'Speaker_gender', 'Speaker_birth', 'Topic 1', 'Confidence 1', 'Topic 2', 'Confidence 2', 'Topic 3', 'Confidence 3']


In [173]:
topic_cols = ['Topic 1', 'Confidence 1', 'Topic 2', 'Confidence 2', 'Topic 3', 'Confidence 3']
all_cols = ['Text_ID'] + topic_cols
filtered = corpus_df.loc[corpus_df[topic_cols].notna().any(axis=1), all_cols]

In [174]:
filtered.head()

,Text_ID,Topic 1,Confidence 1,Topic 2,Confidence 2,Topic 3,Confidence 3
163732,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Parliamentary Oversight and Investigative Comm...,0.62
163733,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Parliamentary Oversight and Investigative Comm...,0.62
163734,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Parliamentary Oversight and Investigative Comm...,0.62
163735,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Parliamentary Oversight and Investigative Comm...,0.62
163736,ParlaMint-AT_2021-03-08-027-XXVII-NRSITZ-00088,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Parliamentary Oversight and Investigative Comm...,0.62


In [3]:
filtered.to_csv("./data/corpus_topics_filtered.csv", index=False)

NameError: name 'filtered' is not defined

# Clean topics

In [3]:
corpus_df = pd.read_csv('data/corpus_topics.csv', index_col=0)
corpus_df.head()

/tmp/ipykernel_8077/1094297537.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  corpus_df = pd.read_csv('data/corpus_topics.csv', index_col=0)


,ID,text,Text_ID,Title,Date,Body,Term,Session,Meeting,Sitting,...,Speaker_ID,Speaker_name,Speaker_gender,Speaker_birth,Topic 1,Confidence 1,Topic 2,Confidence 2,Topic 3,Confidence 3
index,,,,,,,,,,,,,,,,,,,,,
0,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,eröffnen 45 Sitzung nationalrates verhindern m...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,Sitting 45,...,PAD_02822,"Kopf, Karlheinz",M,1957,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Legal Aid and Access to Justice,0.52
1,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,hinsichtlich einlangt verhandlungsnständ zuwei...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,Sitting 45,...,PAD_02822,"Kopf, Karlheinz",M,1957,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Legal Aid and Access to Justice,0.52
2,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,Sitzung nationalrates berufen Donnerstag 23 Ok...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,Sitting 45,...,PAD_02822,"Kopf, Karlheinz",M,1957,Anti-Corruption Enforcement,0.95,Judicial Independence,0.77,Legal Aid and Access to Justice,0.52
3,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049_d...,ehrt n en abordnet vizekanzl wünschen schön en...,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,"Sitting Number 49, Legislative period XXV, Wed...",2014-11-19,Lower house,Legislative period XXV,-,-,Sitting 49,...,PAD_00145,"Bures, Doris",F,1962,Family Welfare,0.95,Government Reform,0.78,Legislation and Policy,0.62
4,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049_d...,Sitzung Bundeskanzleramt Vertretung Mitglied B...,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,"Sitting Number 49, Legislative period XXV, Wed...",2014-11-19,Lower house,Legislative period XXV,-,-,Sitting 49,...,PAD_00145,"Bures, Doris",F,1962,Family Welfare,0.95,Government Reform,0.78,Legislation and Policy,0.62


In [4]:
valid_topics = [
    "Hospital Funding Models (federal vs. provincial responsibilities)",
    "Health Insurance System (ÖGK and other insurers)",
    "Medical Workforce Shortage (rural access and doctor quotas)",
    "Curriculum Reform (digital skills, civic education)",
    "Teacher Recruitment and Pay (regional disparities)",
    "Inclusive Education (support for children with disabilities or language barriers)",
    "University Autonomy (freedom in hiring and budgeting)",
    "Research Funding Allocation (FFG, FWF grant systems)",
    "Access and Tuition (student fees for non-EU nationals)",
    "Conscription Reform (debate over compulsory service)",
    "Cybersecurity Capabilities (military defense in the digital space)",
    "Disaster Relief Operations (Bundesheer role in floods, wildfires)",
    "Counterterrorism Laws (post-2020 Vienna attack measures)",
    "Police Use of Force (body cams, accountability mechanisms)",
    "Border Control and Migration Policing (EU Schengen concerns)",
    "Judicial Independence (selection of judges and prosecutors)",
    "Anti-Corruption Enforcement (high-profile political cases)",
    "Legal Aid and Access to Justice (for low-income citizens)",
    "Tax Reform (income tax brackets, corporate tax adjustments)",
    "Public Debt Management (compliance with EU fiscal rules)",
    "Digital Taxation (taxing tech giants like Google and Amazon)",
    "Austria's Neutrality (debates around NATO relations)",
    "EU Enlargement Policy (position on Balkans integration)",
    "Development Cooperation (aid to Africa and conflict zones)",
    "Minimum Wage Debates (sectoral collective agreements)",
    "Short-Time Work Programs (Kurzarbeit, especially post-COVID)",
    "Skilled Labor Immigration (Red-White-Red Card system)",
    "Pension Age and Sustainability (aging population pressure)",
    "Family Allowances and Child Benefits (Familienbeihilfe)",
    "Basic Income Proposals (pilot discussions and party platforms)",
    "Carbon Pricing System (CO₂ tax and national emissions trading)",
    "Nature Conservation Laws (protection of Alpine biodiversity)",
    "Waste Management and Circular Economy (plastic bans, recycling targets)",
    "Renewable Energy Expansion (wind, solar, hydro)",
    "Energy Grid Modernization (smart grid and interconnectivity)",
    "Public Transport Investment",
    "EU Agricultural Subsidies",
    "Sustainable Farming Practices",
    "Forest Management and Wildfire Prevention",
    "Urban-Rural Infrastructure Gap",
    "Local Government Financing",
    "Housing Policy at Local Level",
    "Public Broadcasting Regulation",
    "Support for Creative Industries",
    "Cultural Heritage Preservation",
    "Parliamentary Oversight and Investigative Committees",
    "Legislative Procedure and Transparency",
    "Political Accountability and Scandals"
]

In [27]:
topic_cols = ['Topic 1', 'Topic 2', 'Topic 3']
invalid_mask = corpus_df[topic_cols].apply(
    lambda row: any(t not in valid_topics for t in row if pd.notna(t)),
    axis=1
)

invalid_topics_df = corpus_df[invalid_mask]

In [28]:
invalid_topics_df

,ID,text,Text_ID,Title,Date,Body,Term,Session,Meeting,Sitting,...,Speaker_ID,Speaker_name,Speaker_gender,Speaker_birth,Topic 1,Confidence 1,Topic 2,Confidence 2,Topic 3,Confidence 3
index,,,,,,,,,,,,,,,,,,,,,


In [29]:
corpus_df.shape

(231413, 30)

In [8]:
topic_columns = ['Topic 1', 'Topic 2', 'Topic 3']

unique_topics = set(
    corpus_df[topic_columns].values.ravel()
) - {None, float('nan')}

unique_topics = {t for t in unique_topics if pd.notna(t)}

print(f"Found {len(unique_topics)} unique topics.")
for topic in sorted(unique_topics):
    print(topic)


Found 862 unique topics.
Abolition of Politician Privileges
Abortion
Abortion Rights
Abzeichengesetz
Accessibility and Inclusivity
Accountability Mechanisms
Accountability and Oversight
Accountability and Transparency
Accountability in Government
Accountability in Public Administration
Administrative Reform
Agricultural Development
Agricultural Policy
Agriculture
Agriculture Policy
Agriculture and Food Security
Ambulance Flights and Border Regions
Animal Welfare
Anti-Communist Sentiment
Anti-Corruption Enforcement
Anti-Doping in Sports
Anti-Nuclear Energy Policy
Anti-Nuclear Policy
Anti-Semitism
Anti-Stalking Laws
Appreciation and Respect for Colleagues
Arms Control and Non-Proliferation
Artistic Freedom
Asylum Policy
Asylum Seeker Policies
Atomic Energy
Austrian Federal Government
Austrian National Assembly Procedure
Austrian Politics
Austrian Security Policy
Banking Regulation
Banking and Financial Regulation
Bauwesen (Construction Industry)
Bergbau
Berichterstattung (Reporting)
Bibl

In [17]:
import spacy
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

nlp = spacy.load("en_core_web_md")

raw_topics = pd.unique(invalid_topics_df[['Topic 1', 'Topic 2', 'Topic 3']].values.ravel())
raw_topics = [t for t in raw_topics if pd.notna(t)]

raw_embeddings = [nlp(text).vector for text in raw_topics]
valid_embeddings = [nlp(text).vector for text in valid_topics]

scores = cosine_similarity(raw_embeddings, valid_embeddings)

# match raw topics to closest valid topic
matches = []
for i, raw_topic in enumerate(raw_topics):
    best_idx = scores[i].argmax()
    matches.append({
        "raw_topic": raw_topic,
        "matched_topic": valid_topics[best_idx],
        "score": scores[i][best_idx]
    })

df_matches = pd.DataFrame(matches).sort_values(by="score", ascending=False)


In [18]:
df_matches

,raw_topic,matched_topic,score
55,Public Broadcasting Regulation,Public Broadcasting Regulation,1.0
244,Political Scandals and Ethics,Political Accountability and Scandals,1.0
21,Parliamentary Oversight and Investigative Comm...,Parliamentary Oversight and Investigative Comm...,1.0
781,Cultural Heritage Preservation,Cultural Heritage Preservation,1.0
376,Parliamentary Procedure and Ethics,Legislative Procedure and Transparency,1.0
...,...,...,...
777,Parlamentarische Geschäfte,Hospital Funding Models (federal vs. provincia...,0.0
841,Finanzausgleichsgesetz,Hospital Funding Models (federal vs. provincia...,0.0
342,Frauenpolitik,Hospital Funding Models (federal vs. provincia...,0.0
343,Politische Gerechtigkeit,Hospital Funding Models (federal vs. provincia...,0.0


In [20]:
topic_mapping = dict(zip(df_matches['raw_topic'], df_matches['matched_topic']))
topic_mapping

{'Public Broadcasting Regulation': 'Public Broadcasting Regulation',
 'Political Scandals and Ethics': 'Political Accountability and Scandals',
 'Parliamentary Oversight and Investigative Committees': 'Parliamentary Oversight and Investigative Committees',
 'Cultural Heritage Preservation': 'Cultural Heritage Preservation',
 'Parliamentary Procedure and Ethics': 'Legislative Procedure and Transparency',
 'Parliamentary Procedure and Oversight': 'Legislative Procedure and Transparency',
 'Parliamentary Procedure and Reform': 'Legislative Procedure and Transparency',
 'Committee Procedure and Oversight': 'Legislative Procedure and Transparency',
 'Parliamentary Procedure and Governance': 'Legislative Procedure and Transparency',
 'Legislative Procedure and Transparency': 'Legislative Procedure and Transparency',
 'Parliamentary Procedures and Oversight': 'Legislative Procedure and Transparency',
 'Parliamentary Procedures and Leadership': 'Legislative Procedure and Transparency',
 'Legis

In [21]:
topic_cols = ['Topic 1', 'Topic 2', 'Topic 3']
corpus_df[topic_cols] = corpus_df[topic_cols].replace(topic_mapping)

In [22]:
corpus_df

,ID,text,Text_ID,Title,Date,Body,Term,Session,Meeting,Sitting,...,Speaker_ID,Speaker_name,Speaker_gender,Speaker_birth,Topic 1,Confidence 1,Topic 2,Confidence 2,Topic 3,Confidence 3
index,,,,,,,,,,,,,,,,,,,,,
0,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,eröffnen 45 Sitzung nationalrates verhindern m...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,Sitting 45,...,PAD_02822,"Kopf, Karlheinz",M,1957,Anti-Corruption Enforcement (high-profile poli...,0.95,Parliamentary Oversight and Investigative Comm...,0.77,Legal Aid and Access to Justice (for low-incom...,0.52
1,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,hinsichtlich einlangt verhandlungsnständ zuwei...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,Sitting 45,...,PAD_02822,"Kopf, Karlheinz",M,1957,Anti-Corruption Enforcement (high-profile poli...,0.95,Parliamentary Oversight and Investigative Comm...,0.77,Legal Aid and Access to Justice (for low-incom...,0.52
2,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045_d...,Sitzung nationalrates berufen Donnerstag 23 Ok...,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,"Sitting Number 45, Legislative period XXV, Wed...",2014-10-22,Lower house,Legislative period XXV,-,-,Sitting 45,...,PAD_02822,"Kopf, Karlheinz",M,1957,Anti-Corruption Enforcement (high-profile poli...,0.95,Parliamentary Oversight and Investigative Comm...,0.77,Legal Aid and Access to Justice (for low-incom...,0.52
3,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049_d...,ehrt n en abordnet vizekanzl wünschen schön en...,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,"Sitting Number 49, Legislative period XXV, Wed...",2014-11-19,Lower house,Legislative period XXV,-,-,Sitting 49,...,PAD_00145,"Bures, Doris",F,1962,Family Allowances and Child Benefits (Familien...,0.95,Parliamentary Oversight and Investigative Comm...,0.78,"Tax Reform (income tax brackets, corporate tax...",0.62
4,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049_d...,Sitzung Bundeskanzleramt Vertretung Mitglied B...,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,"Sitting Number 49, Legislative period XXV, Wed...",2014-11-19,Lower house,Legislative period XXV,-,-,Sitting 49,...,PAD_00145,"Bures, Doris",F,1962,Family Allowances and Child Benefits (Familien...,0.95,Parliamentary Oversight and Investigative Comm...,0.78,"Tax Reform (income tax brackets, corporate tax...",0.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231408,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066_...,davor mögen weidinr Wort melden bitte,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066,"Sitting Number 66, Legislative period XXVI, We...",2019-03-27,Lower house,Legislative period XXVI,-,-,Sitting 66,...,PAD_00145,"Bures, Doris",F,1962,Parliamentary Oversight and Investigative Comm...,0.93,Legislative Procedure and Transparency,0.81,Energy Grid Modernization (smart grid and inte...,0.68
231409,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066_...,schätzen wert Kolleginn wert kollen wissen Hau...,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066,"Sitting Number 66, Legislative period XXVI, We...",2019-03-27,Lower house,Legislative period XXVI,-,-,Sitting 66,...,PAD_02331,"Weidinger, Peter",M,1977,Parliamentary Oversight and Investigative Comm...,0.93,Legislative Procedure and Transparency,0.81,Energy Grid Modernization (smart grid and inte...,0.68
231410,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066_...,Wort niemand melden Debatte schließen weisen A...,ParlaMint-AT_2019-03-27-026-XXVI-NRSITZ-00066,"Sitting Number 66, Legislative period XXVI, We...",2019-03-27,Lower house,Legislative period XXVI,-,-,Sitting 66,...,PAD_00145,"Bures, Doris",F,1962,Parliamentary Oversight and Investigative Comm...,0.93,Legislative Procedure and Transparency,0.81,Energy Grid Modernization (smart grid and inte...,0.68


In [23]:
corpus_df.to_csv("./data/corpus_cleaned.csv", index=False)

In [24]:
topic_cols = ['Topic 1', 'Confidence 1', 'Topic 2', 'Confidence 2', 'Topic 3', 'Confidence 3']
all_cols = ['Text_ID'] + topic_cols
filtered = corpus_df.loc[corpus_df[topic_cols].notna().any(axis=1), all_cols]

In [25]:
filtered.head()

,Text_ID,Topic 1,Confidence 1,Topic 2,Confidence 2,Topic 3,Confidence 3
index,,,,,,,
0,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,Anti-Corruption Enforcement (high-profile poli...,0.95,Parliamentary Oversight and Investigative Comm...,0.77,Legal Aid and Access to Justice (for low-incom...,0.52
1,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,Anti-Corruption Enforcement (high-profile poli...,0.95,Parliamentary Oversight and Investigative Comm...,0.77,Legal Aid and Access to Justice (for low-incom...,0.52
2,ParlaMint-AT_2014-10-22-025-XXV-NRSITZ-00045,Anti-Corruption Enforcement (high-profile poli...,0.95,Parliamentary Oversight and Investigative Comm...,0.77,Legal Aid and Access to Justice (for low-incom...,0.52
3,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,Family Allowances and Child Benefits (Familien...,0.95,Parliamentary Oversight and Investigative Comm...,0.78,"Tax Reform (income tax brackets, corporate tax...",0.62
4,ParlaMint-AT_2014-11-19-025-XXV-NRSITZ-00049,Family Allowances and Child Benefits (Familien...,0.95,Parliamentary Oversight and Investigative Comm...,0.78,"Tax Reform (income tax brackets, corporate tax...",0.62


In [26]:
filtered.to_csv("./data/corpus_topics_cleaned_filtered.csv", index=False)